# Eartrainer C++ Core — Notebook Smoke Test
This notebook sets up import paths, imports the package, and runs a small end-to-end session.

In [1]:
# Path setup so Python can find the compiled _earcore module and the Python package
import sys, os
from pathlib import Path

def add_paths():
    cwd = Path.cwd()
    # Walk up to locate eartrainer/eartrainer_Cpp/{build,python}
    for base in [cwd, *cwd.parents]:
        build = base / 'eartrainer' / 'eartrainer_Cpp' / 'build'
        py = base / 'eartrainer' / 'eartrainer_Cpp' / 'python'
        added = False
        if build.exists():
            sys.path.insert(0, str(build))
            added = True
        if py.exists():
            sys.path.insert(0, str(py))
            added = True
        if added:
            return build, py
    return None, None

build_path, py_path = add_paths()
print('Using build path:', build_path)
print('Using python path:', py_path)


Using build path: /Users/itamarshamir/Projects/ear_trainer/eartrainer/eartrainer_Cpp/build
Using python path: /Users/itamarshamir/Projects/ear_trainer/eartrainer/eartrainer_Cpp/python


In [2]:
# Imports
from eartrainer.session_engine import SessionEngine
from eartrainer.models import SessionSpec, ResultMetrics, ResultReport, SessionSummary, TypedPayload

engine = SessionEngine()
print('Capabilities:', engine.capabilities())


Capabilities: {'assists': ['Replay', 'GuideTone', 'TempoDown', 'PathwayHint'], 'drills': ['note', 'chord'], 'version': 'v1'}


In [3]:
# Run a tiny note session end-to-end
spec = SessionSpec(
    version='v1',
    drill_kind='note',
    key='C major',
    range=[48, 72],
    tempo_bpm=90,
    n_questions=3,
    generation='adaptive',
    assistance_policy={'GuideTone': 1, 'Replay': 2},
    sampler_params={},
    seed=123,
)

session_id = engine.create_session(spec)
print('Session ID:', session_id)

results = []
while True:
    nxt = engine.next_question(session_id)
    # SessionSummary has attribute 'results'; QuestionBundle has 'question_id'
    if hasattr(nxt, 'results'):
        print('Session complete. Totals:', nxt.totals)
        break

    print('Q:', nxt.question_id, 'type:', nxt.question.type, 'payload:', nxt.question.payload)
    # Pull a guide tone assist if available
    try:
        assist = engine.assist(session_id, nxt.question_id, 'GuideTone')
        print('Assist:', assist.kind, assist.ui_delta)
    except Exception as e:
        print('Assist error:', e)

    # UI owns correctness; here we submit the provided correct answer
    metrics = ResultMetrics(rt_ms=900, attempts=1, assists_used={})
    report = ResultReport(
        question_id=nxt.question_id,
        final_answer=TypedPayload(type=nxt.correct_answer.type, payload=dict(nxt.correct_answer.payload)),
        correct=True,
        metrics=metrics,
        client_info={'source': 'notebook'},
    )
    engine.submit_result(session_id, report)


Session ID: sess-1
Q: q-001 type: note payload: {'degree': 4, 'midi': 55, 'tonic_midi': 60}
Assist: GuideTone {'message': 'Guide tone provided'}
Q: q-002 type: note payload: {'degree': 6, 'midi': 59, 'tonic_midi': 60}
Assist: GuideTone {'message': 'Guide tone provided'}
Q: q-003 type: note payload: {'degree': 2, 'midi': 64, 'tonic_midi': 60}
Assist: GuideTone {'message': 'Guide tone provided'}
Session complete. Totals: {'avg_rt_ms': 900, 'correct': 3, 'incorrect': 0}


In [4]:
# Try a chord question (single step)
spec_chord = SessionSpec(
    drill_kind='chord', n_questions=1, generation='eager', seed=42, assistance_policy={'Replay': 1}
)
sid = engine.create_session(spec_chord)
q = engine.next_question(sid)
print('Chord Q:', q.question_id, q.question.type, q.prompt.modality if q.prompt else None)
metrics = ResultMetrics(rt_ms=700, attempts=1, assists_used={})
engine.submit_result(sid, ResultReport(
    question_id=q.question_id,
    final_answer=TypedPayload(type=q.correct_answer.type, payload=dict(q.correct_answer.payload)),
    correct=True, metrics=metrics, client_info={}))
print('Chord summary:', engine.next_question(sid).totals)


Chord Q: q-001 chord midi
Chord summary: {'avg_rt_ms': 700, 'correct': 1, 'incorrect': 0}


In [5]:
# Helpers and note sampling utilities
NOTE_NAMES = ['C', 'C#', 'D', 'D#', 'E', 'F', 'F#', 'G', 'G#', 'A', 'A#', 'B']

def midi_to_name(midi: int) -> str:
    note = NOTE_NAMES[midi % 12]
    octave = midi // 12 - 1
    return f"{note}{octave}"

def wrap_scale_degree(degree: int) -> int:
    idx = degree % 7
    if idx < 0:
        idx += 7
    return idx + 1

def sample_note_questions(key: str = 'C major', n: int = 20, seed: int = 101):
    spec = SessionSpec(
        drill_kind='note',
        key=key,
        n_questions=n,
        generation='eager',
        assistance_policy={},
        seed=seed,
    )
    session_id = engine.create_session(spec)
    rows = []
    for _ in range(n):
        bundle = engine.next_question(session_id)
        if isinstance(bundle, SessionSummary):
            break
        raw = bundle.question.payload
        rows.append({
            'question_id': bundle.question_id,
            'raw': raw,
            'degree_raw': raw['degree'],
            'note_name': midi_to_name(raw['midi']),
        })
        metrics = ResultMetrics(rt_ms=600, attempts=1, assists_used={})
        report = ResultReport(
            question_id=bundle.question_id,
            final_answer=bundle.correct_answer,
            correct=True,
            metrics=metrics,
            client_info={'source': 'notebook-note-sample'},
        )
        engine.submit_result(session_id, report)
    return rows

note_key = 'B major'
note_seed = 101
note_samples = sample_note_questions(key=note_key, n=20, seed=note_seed)
print(f'Key: {note_key} (seed={note_seed})')
for item in note_samples:
    print(f"{item['question_id']}: degree {item['degree_raw']} (raw={item['degree_raw']}) -> {item['note_name']} | payload={item['raw']}")



Key: B major (seed=101)
q-001: degree 4 (raw=4) -> F#4 | payload={'degree': 4, 'midi': 66, 'tonic_midi': 59}
q-002: degree 1 (raw=1) -> C#4 | payload={'degree': 1, 'midi': 61, 'tonic_midi': 59}
q-003: degree 0 (raw=0) -> B4 | payload={'degree': 0, 'midi': 71, 'tonic_midi': 59}
q-004: degree 3 (raw=3) -> E3 | payload={'degree': 3, 'midi': 52, 'tonic_midi': 59}
q-005: degree 4 (raw=4) -> F#3 | payload={'degree': 4, 'midi': 54, 'tonic_midi': 59}
q-006: degree 3 (raw=3) -> E4 | payload={'degree': 3, 'midi': 64, 'tonic_midi': 59}
q-007: degree 5 (raw=5) -> G#4 | payload={'degree': 5, 'midi': 68, 'tonic_midi': 59}
q-008: degree 6 (raw=6) -> A#3 | payload={'degree': 6, 'midi': 58, 'tonic_midi': 59}
q-009: degree 1 (raw=1) -> C#3 | payload={'degree': 1, 'midi': 49, 'tonic_midi': 59}
q-010: degree 0 (raw=0) -> B3 | payload={'degree': 0, 'midi': 59, 'tonic_midi': 59}
q-011: degree 2 (raw=2) -> D#4 | payload={'degree': 2, 'midi': 63, 'tonic_midi': 59}
q-012: degree 6 (raw=6) -> A#3 | payload={'de

In [9]:
# Chord sampling preview
def sample_chord_questions(key: str = 'C major', n: int = 60, seed: int = 202):
    spec = SessionSpec(
        drill_kind='chord',
        key=key,
        n_questions=n,
        generation='eager',
        assistance_policy={},
        seed=seed,
    )
    session_id = engine.create_session(spec)
    rows = []
    for _ in range(n):
        bundle = engine.next_question(session_id)
        if isinstance(bundle, SessionSummary):
            break
        raw = bundle.question.payload
        voicing_names = [midi_to_name(m) for m in raw['voicing_midi']]
        rows.append({
            'question_id': bundle.question_id,
            'degree': wrap_scale_degree(raw['root_degree']),
            'quality': raw.get('quality'),
            'voicing': voicing_names,
            'raw': raw,
        })
        metrics = ResultMetrics(rt_ms=600, attempts=1, assists_used={})
        report = ResultReport(
            question_id=bundle.question_id,
            final_answer=bundle.correct_answer,
            correct=True,
            metrics=metrics,
            client_info={'source': 'notebook-chord-sample'},
        )
        engine.submit_result(session_id, report)
    return rows

chord_key = 'C major'
chord_seed = 202
chord_samples = sample_chord_questions(key=chord_key, n=60, seed=chord_seed)
print(f'Key: {chord_key} (seed={chord_seed})')
for item in chord_samples:
    voicing_str = ', '.join(item['voicing'])
    if item['degree'] == 7:
        print(f"{item['question_id']}: degree {item['degree']} ({item['quality']}) -> {voicing_str}")
    #print(f"    raw={item['raw']}")



Key: C major (seed=202)
q-001: degree 7 (diminished) -> B1, F3, B3, D4
q-008: degree 7 (diminished) -> B1, B3, D4, F4
q-014: degree 7 (diminished) -> B1, B3, D4, F4, B4
q-016: degree 7 (diminished) -> B1, B3, D4, F4, B4
q-021: degree 7 (diminished) -> B1, B3, D4, F4
q-025: degree 7 (diminished) -> B1, F3, B3, D4
q-039: degree 7 (diminished) -> B1, B3, D4, F4
q-041: degree 7 (diminished) -> B1, F3, B3, D4
q-049: degree 7 (diminished) -> B1, B3, D4, F4


In [ ]:
# Interval sampling preview
def sample_interval_questions(key: str = 'C major', n: int = 20, seed: int = 303):
    spec = SessionSpec(
        drill_kind='interval',
        key=key,
        n_questions=n,
        generation='eager',
        assistance_policy={},
        seed=seed,
    )
    session_id = engine.create_session(spec)
    rows = []
    for _ in range(n):
        bundle = engine.next_question(session_id)
        if isinstance(bundle, SessionSummary):
            break
        payload = bundle.question.payload
        bottom_name = midi_to_name(payload['bottom_midi'])
        top_name = midi_to_name(payload['top_midi'])
        rows.append({
            'question_id': bundle.question_id,
            'bottom_deg': payload['bottom_degree'],
            'top_deg': payload['top_degree'],
            'bottom_midi': bottom_name,
            'top_midi': top_name,
            'raw': payload,
            'answer': bundle.correct_answer.payload,
        })
        metrics = ResultMetrics(rt_ms=650, attempts=1, assists_used={})
        report = ResultReport(
            question_id=bundle.question_id,
            final_answer=bundle.correct_answer,
            correct=True,
            metrics=metrics,
            client_info={'source': 'notebook-interval-sample'},
        )
        engine.submit_result(session_id, report)
    return rows

interval_key = 'C major'
interval_seed = 303
interval_samples = sample_interval_questions(key=interval_key, n=20, seed=interval_seed)
print(f'Key: {interval_key} (seed={interval_seed})')
for item in interval_samples:
    bottom = item['bottom_midi']
    top = item['top_midi']
    answer = item['answer']
    answer_name = answer.get('name') if isinstance(answer, dict) else answer
    print(f"{item['question_id']}: bottom deg {item['bottom_deg']} -> {bottom}, top deg {item['top_deg']} -> {top} | interval {answer_name} | raw={item['raw']}")

